In [5]:
import pandas as pd
from utils import util

In [6]:
def get_seq_pred_by_FBF(seq_label_dir, window_size, fbf_pred_dir):
    
    seq_label_df = pd.read_csv(seq_label_dir)
    fbf_pred_df = pd.read_csv(fbf_pred_dir)
    seq_res_by_fbf = pd.DataFrame(columns=['img_path', 'emo_gt', 'emo_pred', 'ave_emo_pos'])
    
    for i, row in enumerate(seq_label_df.itertuples()):
        img_path = row.img_path
        emo_gt = row.emotion
        img_path_list = util.get_sequence_img_path(current_img_path=img_path, window_size=window_size, current_position='head')
        
        # extract fbf_pred_df by img_path_list
        _fbf_pred_df = fbf_pred_df[fbf_pred_df['img_path'].isin(img_path_list)]
        if len(_fbf_pred_df) == 0:
            # ave_emo_pos = 0
            continue
        else:
            ave_emo_pos = _fbf_pred_df['emo_pos'].mean()
            
        emo_pred = 0
        if ave_emo_pos >= 0.5:
            emo_pred = 1
        
        seq_res_by_fbf.loc[i] = [img_path, emo_gt, emo_pred, ave_emo_pos]
        
    seq_res_by_fbf = seq_res_by_fbf.reset_index(drop=True)
    
    return seq_res_by_fbf
        

In [7]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_auc_score, precision_recall_curve, auc

def get_metrics_from_pred_df(pred_df, target_emo):
    gt_list = []
    if target_emo == "comfort":
        # copy pred["emo_gt"] to gt_list
        gt_list = pred_df["emo_gt"].copy()
        # replace 2 to 0
        gt_list = gt_list.replace(2, 0)
    elif target_emo == "discomfort":
        # copy pred["emo_gt"] to gt_list
        gt_list = pred_df["emo_gt"].copy()
        # replace 1 to 0
        gt_list = gt_list.replace(1, 0)
        # replace 2 to 1
        gt_list = gt_list.replace(2, 1)
        
    # culc metrics gt_list and pred["emo_pred"]
    acc = accuracy_score(gt_list, pred_df["emo_pred"])
    precision = precision_score(gt_list, pred_df["emo_pred"])
    recall = recall_score(gt_list, pred_df["emo_pred"])
    f1 = f1_score(gt_list, pred_df["emo_pred"])
    roc_auc = roc_auc_score(gt_list, pred_df["emo_pred"])
    pre, rec, _ = precision_recall_curve(gt_list, pred_df["emo_pred"])
    pr_auc = auc(rec, pre)
    
    metrics = pd.DataFrame([precision, recall, f1, acc, roc_auc, pr_auc]).T
    metrics.columns = ["precision", "recall", "f1", "accuracy", "roc_auc", "pr_auc"]
    
    return metrics

In [18]:
target_emo = "discomfort"
run_name = "2_d_g_ylogits"
epoch = 3

seq_label_dir = "/mnt/iot-qnap3/mochida/medical-care/emotionestimation/data/labels/PIMD_A/discomfort_nomixed_seq_labels_wsize30-ssize5.csv"
fbf_pred_dir = f"/mnt/iot-qnap3/mochida/medical-care/emotionestimation/reports/PIMD_A/{run_name}/epoch{epoch}/{target_emo}_pred_all.csv"

df = get_seq_pred_by_FBF(seq_label_dir, window_size=30, fbf_pred_dir=fbf_pred_dir)

In [19]:
met = get_metrics_from_pred_df(df, target_emo="discomfort")

In [20]:
met

,precision,recall,f1,accuracy,roc_auc,pr_auc
0,0.5625,0.1843,0.277635,0.508741,0.516881,0.582316


In [21]:
#! ファイル名の変更を忘れずに
df.to_csv(f"/mnt/iot-qnap3/mochida/medical-care/emotionestimation/reports/PIMD_A/{run_name}/epoch{epoch}/{target_emo}_seq_pred_by_fbf.csv", index=False)
met.to_csv(f"/mnt/iot-qnap3/mochida/medical-care/emotionestimation/reports/PIMD_A/{run_name}/epoch{epoch}/{target_emo}_seq_pred_by_fbf_metrics(micro_ave).csv", index=False)